<a href="https://colab.research.google.com/github/jonhsel/colabs_gogle/blob/main/BI_sessoes_juri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUTOMAÇÃO DA ETAPA T - TRANSFORMAÇÃO DO ETL

Etapa realizada após a extração dos dados no sistema SIMP!

## Bibliotecas necessárias

In [ ]:
import pandas as pd
import glob

## Criação de rotina para:



1.   Carregar os arquivos .xls;
2.   Concatenar os diversos arquivos em um único arquivo;
3.   Transformar o arquivo em um objeto do tipo  Dataframe;

In [ ]:
#Função para carregar os arquivos
def painel_bi_sessaojuri(caminho_diretorio):
    """
        - Carrega todos os arquivos xLs em um determinado diretório e retorna um dataframe;
        - Concatena-os em um único Dataframe
        - Salve me um arquivo .csv

        ARGS:
            caminho_diretorio : O caminho completo para o diretorio onde os arquivos estão;
            nome_arquivo_saida: O nome do arquivo CSV de saida
    """
    #Encontra os arquivos XLS no diretório
    arquivos_xls = glob.glob(caminho_diretorio + "/*.xls")

    #Criar uma lisa para armazenar os Dataframes
    dataframes= []

    #Itera sobre cada arquivo e cria um DataFrame
    for arquivo in arquivos_xls:
        df = pd.read_excel(arquivo)
        dataframes.append(df)

    #Concatena todos os DataFrames em um único Dataframe
    df_concatenado = pd.concat(dataframes, ignore_index=True)

    return df_concatenado

In [ ]:
caminho = "/content/drive/MyDrive/CAOJURI/08 - agosto"

In [ ]:
df_resultado = painel_bi_sessaojuri(caminho)

In [ ]:
df_resultado.head()

In [ ]:
df_resultado.columns

## Separação do "Movimento"

In [ ]:
#Divisão
df_resultado['Movimento_split'] = df_resultado.Movimento.str.split('-->')
df_resultado['Movimento_split'].head()

In [ ]:
# prompt: gerar código que selecione o primeiro elemento da série df_resultado['Movimento_split] e insira na série df_resultado['Movimento_final]
df_resultado['Movimento_final'] = df_resultado['Movimento_split'].str[0]
df_resultado.head()

##  Engenharia de Características
Criação da dimensão 'Fase do Movimento' com a criação das variáveis "1ª Fase" e "Sessão do Tribunal do Júri".

In [ ]:
# Cria uma série chamada df_resultado['Fase_movimento']; se df_resultado['Movimento_final'] == 'ABSOLVIÇÃO' ou 'ABSOLVIÇÃO A PEDIDO' ou 'CONDENAÇÃO' ou 'CONDENAÇÃO PARCIAL' ou 'DESCLASSIFICAÇÃO' ou 'EXTINÇÃO DE PUNIBILIDADE', a serie df_resultado['Fase_movimento']  == 'Sessão do Tribunal do Júri', se não df_resultado['Fase_movimento']  == '1ª Fase'

df_resultado['Fase_movimento'] = ''
for i in range(0,len(df_resultado)):
  if df_resultado['Movimento_final'][i] == 'ABSOLVIÇÃO' or df_resultado['Movimento_final'][i] == 'ABSOLVIÇÃO A PEDIDO DO MP' or df_resultado['Movimento_final'][i] == 'CONDENAÇÃO' or df_resultado['Movimento_final'][i] == 'CONDENAÇÃO PARCIAL' or df_resultado['Movimento_final'][i] == 'DESCLASSIFICAÇÃO' or df_resultado['Movimento_final'][i] == 'EXTINÇÃO DE PUNIBILIDADE':
    df_resultado['Fase_movimento'][i] = 'Sessão do Tribunal do Júri'
  else:
    df_resultado['Fase_movimento'][i] = '1ª Fase'

df_resultado.head(20)

In [ ]:
df_resultado.info()

## Reordenação das colunas

As colunas são reordenadas para mesma sequência em que aparecem na base de dados do BI

In [ ]:
df_final = df_resultado[['Protocolo','Data','Movimento_final','Fase_movimento','Assunto','Promotor','Promotoria','Comarca']]
df_final.head()

## Geração do arquivo de saída e  download

In [ ]:
nome_saida = "/content/drive/MyDrive/CAOJURI/08 - agosto/CSVPainelBI-08"

In [ ]:
df_final.to_csv(nome_saida+'.csv', index=False)